In [329]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of custom_classes_defs.hed failed: Traceback (most recent call last):
  File "/Users/bawfeh78/miniconda3/envs/tf2_py310/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/bawfeh78/miniconda3/envs/tf2_py310/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/bawfeh78/miniconda3/envs/tf2_py310/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/bawfeh78/Documents/UIS/Masters/Code/custom_classes_defs/hed.py", line 1, in <module>
    from setup import *
ModuleNotFoundError: No module named 'setup'
]


In [330]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" # all | last | last_expr | none 

In [331]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [332]:
# ============= Import required packaages ==============
import re, time, os

from keras.utils import plot_model

# Import all custom variables and modules
from custom_classes_defs.preprocessing import *
# from custom_classes_defs.Unet_like import *   
# from custom_classes_defs.fnet import *
from custom_classes_defs.hed import *

import keras
import numpy as np

RND_STATE = 247
BATCH_SIZE = 32
keras.utils.set_random_seed(RND_STATE)

INTERACTIVE_SESSION = True

# from tensorflow.data import Dataset as tf_data
# -------------------------------------------------------

In [333]:
pets = Oxford_Pets(
        input_dir = "./data/images/",
        target_dir = "./data/annotations/trimaps/",
        img_size = (160, 160),
        batch_size = BATCH_SIZE
    )

train_dataset, valid_dataset, test_dataset = \
        pets.split_data(seed=RND_STATE)

#for input_path, target_path in zip(pets.input_img_paths[:10], pets.target_img_paths[:10]):
#    print(input_path, "|", target_path)
print(f"training data (size = {len(train_dataset)*BATCH_SIZE})")
print(f"validation data (size = {len(valid_dataset)*BATCH_SIZE})")
print(f"test data (size = {len(test_dataset)*BATCH_SIZE})")
print("Data images tensor:",train_dataset.element_spec[0])
print("Data labels tensor:",train_dataset.element_spec[1])


Number of samples: 7390
training data (size = 672)
validation data (size = 96)
test data (size = 6656)
Data images tensor: TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name=None)
Data labels tensor: TensorSpec(shape=(None, 160, 160, 1), dtype=tf.uint8, name=None)


In [334]:
# Display input image #idx
idx = 19
display(Image(filename=pets.input_img_paths[idx]))

# Display auto-contrast version of corresponding target (per-pixel categories)
img = ImageOps.autocontrast(load_img(pets.target_img_paths[idx]))
display(img)

KeyboardInterrupt: 

In [ ]:

# Model configurations
conf = model_config(
    epochs=30,
    batch_size=BATCH_SIZE,
    shuffle=False,
    scaling=1,
    verbose=1,
    save_path='./data',
    augmentation=False,
    img_shape=pets.img_size,
    target_size=pets.img_size,
    channels_dim=(3,3)
)


es_callbacks = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

callbacks = [
    #keras.callbacks.ModelCheckpoint("oxford_segmentation.keras", save_best_only=True),
    es_callbacks
]

conf.set( validation_data=valid_dataset,  callbacks=callbacks )
conf.set(
    'compile',
    optimizer=keras.optimizers.Adam(1e-4), 
    loss="sparse_categorical_crossentropy"
)
conf.info()

In [ ]:
### Interactive step:
if INTERACTIVE_SESSION:
    train = input("New train session? (y/n): ")
    if train[0].lower()=='y':
        conf.new_training_session = True
    else:
        conf.new_training_session = False
    interact = input("Are you sure, you want to run this session interactively? (y/n): ")
    if interact[0].lower()!='y':
        INTERACTIVE_SESSION = False

### Build model

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Build model', '-'*55))
# m_obj = UNET2D(panel_sizes=[32,64,128,256], model_arch=conf.model_arch)
m_obj = HED2D(model_arch=conf.model_arch)
model = m_obj.build_model()
model.summary()
# plot_model(model, 'm_obj.png',show_shapes=True)
print(f"Total trainable wieghts: {model.count_params():,}")



In [ ]:
#if model.name == 'hed':
#    # Enforce transfer learning
#    for ly in model.layers:
#        if ly.trainable and re.search('block[1-5]_conv', ly.name):
#            ly.trainable = False

### Train the model

In [ ]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.

print("\n\n{}\n\t{}\n{}".format('='*55,f'Train {m_obj.Name} model', '-'*55))

conf.execute_training(
    model, 
    data=train_dataset, 
    saveas=m_obj.Name+'_pets', 
    plot_history=INTERACTIVE_SESSION
)



### Visualize predictions

In [ ]:
# Generate predictions for all images in the validation set

val_preds = model.predict(valid_dataset)

if INTERACTIVE_SESSION:
    pets.display_sample_image(val_preds, image_id=10)
